**Install necessary libraries**

In [1]:
!pip install tensorflow numpy pandas scikit-learn

**Import libraries**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re

**Load the dataset with specified encoding**

In [3]:
file_path = 'test.csv'
data = pd.read_csv(file_path, encoding='latin-1')

**Map sentiment values to integers**

In [4]:
data['sentiment'] = data['sentiment'].map({'positive': 4, 'neutral': 2, 'negative': 0})

**Split the data into train and test sets**

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.2, random_state=42)

**Drop rows with missing values**

In [6]:
# Drop rows with missing values in the train set
X_train = X_train.dropna()
y_train = y_train.dropna()

# Drop rows with missing values in the test set
X_test = X_test.dropna()
y_test = y_test.dropna()

**Convert text data into numerical format**

In [7]:
vectorizer = TfidfVectorizer(max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

**Convert sentiment labels into numerical format**

In [8]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)

**Define LSTM model**

In [9]:
model = Sequential()
model.add(Embedding(input_dim=2000, output_dim=64, input_length=X_train_vec.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

**Compile the model**

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the model**

In [11]:
# Sort the indices of the sparse matrix
X_train_vec.sort_indices()

# Train the model
model.fit(X_train_vec, y_train_encoded, batch_size=64, epochs=5)

Epoch 1/5
45/45 [==============================] - 187s 4s/step - loss: 1.0868 - accuracy: 0.4047
Epoch 2/5
45/45 [==============================] - 214s 5s/step - loss: 1.0852 - accuracy: 0.4107
Epoch 3/5
45/45 [==============================] - 220s 5s/step - loss: 1.0850 - accuracy: 0.4107
Epoch 4/5
45/45 [==============================] - 183s 4s/step - loss: 1.0860 - accuracy: 0.4107
Epoch 5/5
45/45 [==============================] - 183s 4s/step - loss: 1.0856 - accuracy: 0.4107


**Evaluate the model**

In [12]:
# Sort the indices of the sparse matrix
X_test_vec.sort_indices()

# Evaluate the model
loss, accuracy = model.evaluate(X_test_vec, y_test_encoded, verbose=1)
print('Test Accuracy:', accuracy)

23/23 [==============================] - 10s 399ms/step - loss: 1.0957 - accuracy: 0.3801
Test Accuracy: 0.38014185428619385


**Function to predict the sentiment**

In [13]:
def predict_sentiment(text, vectorizer, model, label_encoder):
    # Preprocess the text
    text = preprocess_text(text)
    # Convert text into numerical format
    text_vec = vectorizer.transform([text])
    # Ensure sorted indices
    text_vec.sort_indices()
    # Convert sparse matrix to dense array
    text_vec_dense = text_vec.toarray()
    # Predict sentiment
    pred = model.predict(text_vec_dense)
    # Decode predicted sentiment
    sentiment_label = label_encoder.inverse_transform([np.argmax(pred)])[0]

    if sentiment_label == 4:
        return "Positive"
    elif sentiment_label == 2:
        return "Neutral"
    else:
        return "Negative"


**Function to preprocess the text from user**

In [15]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text


**Function to run the sentiment prediction chatbot**

In [16]:
def run_sentiment_chatbot(vectorizer, model, label_encoder):
    while True:
        # Prompt the user for input
        user_input = input("Enter a statement (type 'exit' or 'quit' to end): ")

        # Check if the user wants to exit
        if user_input.lower() in ['exit', 'quit', 'goodbye', 'bye']:
            print("Goodbye! Exiting the chatbot.")
            break

        # Predict sentiment
        predicted_sentiment = predict_sentiment(user_input, vectorizer, model, label_encoder)

        # Print the predicted sentiment
        print("Predicted Sentiment:", predicted_sentiment)

# Run the sentiment prediction chatbot
run_sentiment_chatbot(vectorizer, model, label_encoder)


Enter a statement (type 'exit' or 'quit' to end): Hi
1/1 [==============================] - 1s 559ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): How are you?
1/1 [==============================] - 0s 151ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): I am very Happy
1/1 [==============================] - 0s 161ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): Fuck you
1/1 [==============================] - 0s 155ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): O my God
1/1 [==============================] - 0s 182ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): AHAHAHAHAHAHAHAHAHAHA
1/1 [==============================] - 0s 153ms/step
Predicted Sentiment: Neutral
Enter a statement (type 'exit' or 'quit' to end): Exit
Goodbye! Exiting the chatbot.
